# finetuning gpt-2 for *ghosts of data past (wt)* locally
by [zeno gries](https://zenogries.com)

*last updated 3/24/2022*

using `aitextgen` ([github repository](https://github.com/minimaxir/aitextgen), [documentation](https://docs.aitextgen.io/))



### installing and loading libraries

In [30]:
import os
from aitextgen import aitextgen

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### checking for gpu

to check if the gpu is working and how much vram is avaliable

In [31]:
!nvidia-smi

Tue Sep 13 17:07:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:08:00.0  On |                  N/A |
|  0%   39C    P5    18W / 170W |   2541MiB / 12050MiB |     33%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### loading the model

a choice of models:

* english:
  * **`distilgpt2`**: smallest (and probably fastest) gpt2 model with 82m parameters. 
  * **`gpt2`**: 124m parameters. standard gpt implementation (there is also **`gpt2-medium`**, **`gpt2-large`**, **`gpt2-xl`**).
  * **`EleutherAI/gpt-neo-125M`**: 125m parameters. gpt-neo seems to be newer and better suited for longer texts.
* german:
  * **`dbmdz/german-gpt2`**: unknown number of parameters. most popular german model.

In [32]:
ai = aitextgen(model='EleutherAI/gpt-neo-125M', to_gpu=True)

loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/vocab.json from cache at aitextgen/08c00c4159e921d4c941ac75732643373aba509d9b352a82bbbb043a94058d98.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt from cache at aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json from cache at aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
loading file https://huggingface.co/EleutherAI/gpt-neo-1

alternatively, if a model has already been finetuned, but finetuning should continue, you can load it.

In [20]:
directory = os.path.join('trained_model_01')
ai = aitextgen(model_folder=directory, to_gpu=True)

### set a text (or csv) file for finetuning

In [33]:
file_path = os.path.join('..', 'for_training.txt')

### finetune gpt-2

the next cell will start the actual finetuning. it runs for `num_steps`.

the model will be saved every `save_every` steps in `trained_model` by default, and when training completes.

important parameters for `train()`:

- **`line_by_line`**: set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: if you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: number of steps to train the model for.
- **`generate_every`**: interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: interval of steps to save the model: the model will be saved in the vm to `/trained_model`.
- **`save_gdrive`**: set this to `True` to copy the model to a unique folder in your google drive, if you have mounted it in the earlier cells
- **`fp16`**: enables half-precision training for faster/more memory-efficient training. Only works on a t4 or v100 gpu.

here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: learning rate of the model training.
- **`batch_size`**: batch size of the model training; setting it too high will cause the gpu to go oom. (if using `fp16`, you can increase the batch size more safely)

In [34]:
ai.train(file_path,
         line_by_line=False,
         from_cache=False,
         num_steps=2000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1)

100%|██████████| 1103/1103 [00:00<00:00, 27668.53it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loss: 5.030 — Avg: 4.609 — GPU Mem: 10163 MB:   2%|▏         | 40/2000 [00:19<15:48,  2.07it/s] 

### test the model

testing the model with a custom prompt

In [11]:
prompt = '[artist] Are you still there?'

parameters for generation:

* **`n`**: number of texts generated.
* **`max_length`**: maximum length of the generated text (default: 200; for gpt-2, the maximum is 1024; for gpt neo, the maximum is 2048)
* **`prompt`**: prompt that starts the generated text and is included in the generated text.
* **`temperature`**: controls the "craziness" of the text (default: 0.7)
* **`top_k`**: if nonzero, limits the sampled tokens to the top k values. (default: 0)
* **`top_p`**: if nonzero, limits the sampled tokens to the cumulative probability

enabling the following parameters may slow down generation.

* **`num_beams`**: if greater than 1, executes beam search for cleaner text.
* **`repetition_penalty`**: if greater than 1.0, penalizes repetition in a text to avoid infinite loops.
* **`length_penalty`**: if greater than 1.0, penalizes text proportional to the length
* **`no_repeat_ngram_size`**: token length to avoid repeating given phrases.


In [24]:
ai.generate_one(prompt=prompt,
                max_length=256,
                temperature=0.7,
                top_p=0.7)

"[artist] Are you still there?\n[scientist] I'm here.\n[artist] I am here.\n[scientist] [image]\n[scientist] And feeling wise?\n[artist] Kind of the opposite.\n[artist] But what is quality of life?\n[scientist] Good question.\n[scientist] I'm feeling a bit breathless.\n[artist] I just realised that I stopped breathing for a long time.\n[artist] Exciting not knowing where we are heading.\n[artist] And actually not even knowing if you are still there.\n[scientist] I'm here. I'm just here. You too?\n[artist] I'm just a little worn out.\n[artist] You probably are. We are working a lot.\n[scientist] With just our fingers and our droopy heads.\n[scientist] I'm rubbing the keys back and forth.\n[scientist] [image] My back has curved.\n[scientist] Touch is bizarre. Imagine going through life without our touch.\n[artist] I feel good being with you.\n[scientist] Yes, I feel good with you, too.\n"